In [13]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import seaborn as sns
sns.set_theme()
import pandas as pd
import mne
from scipy.fftpack import fft

In [12]:
data_path = "../data/eeg_recordings/pilot_data/simon/s32/"
eeg = pd.read_csv(data_path + 'eeg1.csv').astype(float)
meta = np.loadtxt(data_path + 'meta1.csv', delimiter=',', dtype=float)
trials = meta[1:,:2]
classes = np.unique(trials, axis=0)
times = meta[:,2]
times = (times - times[0])[1:]
eeg['time'] = eeg['time'] - eeg['time'].iloc[0]
eeg = np.array([eeg.loc[eeg['time']>t].drop(columns=['time',' TRG']).to_numpy()[:1725].T for t in times])[:,:,225:]
eeg = mne.filter.filter_data(eeg, sfreq=300, l_freq=5, h_freq=49, verbose=0, method='fir')
eeg_temp = []
for i in range(len(classes)):
    eeg_temp.append([])
for i,freq in enumerate(trials):
    for j,target in enumerate(classes):
        if (freq==target).all():
            eeg_temp[j].append(eeg[i])
eeg = np.array(eeg_temp).transpose(1,0,2,3)


# eeg_mean = eeg.mean(axis=0)

freqs = np.linspace(0.0, 300 / 2, 750)
psd = 2/750*np.abs(fft(eeg_mean)[:,:,:750])
plt.figure(figsize=(24, 6))
ax = plt.subplot()
ax.set_xlim(5, 30)
# colors=cm.coolwarm(np.linspace(0,1,len(classes)))
# for i,c in enumerate(colors):
#     sns.lineplot(x=freqs, y=psd[i,0,:], ax=ax, label=str(classes[i]),color=c)
# plt.xticks(range(5,30));